In [1]:
#!/usr/bin/env python
# -*- coding: gb18030 -*-
# @Time    : 2017/7/8 10:50
# @Author  : taoyongbo
# @Site    : 
# @File    : rankPrediction.py
# @desc    :
import csv
import multiprocessing
import os
import sys
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.spatial.distance import cdist
from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.utils import shuffle
from sklearn.externals import joblib
import constant
import utils

logger = utils.get_logger('rankPrediction')

local_weight_path = "D:\structure\\poi-weight.txt"

model_map = {
    "binguanfandian.m": "宾馆饭店",
    "jinrongyinxing": "金融银行",
    "gouwuchangsuo": "购物场所",
    "zhengfujiguan": "政府机关",
    "youzhengdianxin": "邮政电信",
    "yiliaoweisheng": "医疗卫生",
    "xuexiaokeyan": "学校科研",
    "xiuxianyule": "休闲娱乐",
    "xinwenmeiti": "新闻媒体",
    "tiyuchangguan": "体育场馆",
    "qichefuwu": "汽车服务",
    "jiaotongchuxing ": "交通出行",
    "gongsiqiye": "公司企业",
    "diming": "地名",
    "canyinfuwu": "餐饮服务",
    "changguanhuisuo": "场馆会所",
    "fangdichan": "房地产",
    "qita ": "其它",
    "lvyoujingdian.m": "旅游景点"
}
def get_model_map(file_name):

    if file_name in model_map:
        return model_map.get(file_name)
    return None


weight = pd.read_csv(local_weight_path, sep='\t',
                     names=['parentCategory', 'subcategory', 'subCategoryScore', 'tagScore', 'matchCountScore',
                            'gradeScore',
                            'commentScore', 'priceScore', 'areaScore', 'leafCountScore', 'doorCountScore',
                            'parkCountScore', 'innerCountScore',
                            'buildCountScore'], quoting=csv.QUOTE_NONE, encoding='gb18030')


def load_models(model_file_list):
    model_dict = {}
    for model_file in model_file_list:
        file_name = os.path.basename(model_file)

        binguanfandian_model = joblib.load(model_file)
        key = get_model_map(file_name)
        model_dict.__setitem__(key, binguanfandian_model)
    return model_dict

def files_rank_cluster(local_featurePoi_path, output_path=None, cluster_type="multi", column=None):
    global local_featurePoiRank_path

    if not output_path == "":
        local_featurePoiRank_path = output_path

    begin_time = time.time()
    logger.info("all fileList handle process")

    model_file_list, model_dirList = utils.get_files('D:\structure\\featurePoi\model\\')

    #load model
    model_dict = load_models(model_file_list)


    # 读取文件
    fileList, dirList = utils.get_files(local_featurePoi_path)


    logger.info('total file:{num}'.format(num=len(fileList)))
    pool = multiprocessing.Pool(processes=1)

    for file in fileList:
        pool.apply_async(rank_prediction, (file,model_dict,))  #
    pool.close()
    pool.join()

    end_time = time.time()

    logger.info(u"完成用时" + str(end_time - begin_time) + "s")
    logger.info("all fileList handle finished")


def rank_prediction(file,model_dict):
    try:
        featurePoi = pd.read_csv(file,sep='\t',encoding='gb18030',names=['name', 'dataId', 'city', 'parentCategory', 'subCategory', 'brand',
             'categoryScore', 'tagScore', 'matchCountScore', 'gradeScore',
             'commentScore', 'priceScore', 'areaScore', 'leafCountScore', 'doorCountScore',
             'parkCountScore', 'innerCountScore', 'buildCountScore',
             'point', 'keyword', 'matchCount', 'grade', 'comment', 'price', 'area',
             'leafCount', 'doorCount', 'parkCount', 'innerCount', 'buildCount',
             'hotCount', 'hitcount', 'viewcount', 'sumViewOrder', 'SUMWEIGHT', 'LABEL', 'RANK_LEVEL', 'HITCOUNT_LABEL',
             'HITCOUNT_RANK', 'HOTCOUNT_LABEL', 'HOTCOUNT_RANK', 'CITYSIZE'],quoting=csv.QUOTE_NONE)

        parentCategory = featurePoi['parentCategory'][0]
        model = model_dict.get(parentCategory)
        begin_time = time.time()
        W = weight.ix[weight.parentCategory == parentCategory, 2:]
        X = featurePoi.ix[:, 6:18]

        featureValues = np.multiply(X, W)

        featurePoi.loc[:, ('sumWeight1')] = featureValues.sum(axis=1)

        y_train_pred = model.predict(featurePoi.loc[:, ['sumWeight1', 'CITYSIZE']])

        featurePoi['y_train_pred'] = y_train_pred
        featurePoi['pre_rank_level'] = featurePoi['y_train_pred'].apply(func=convert_level)

        print(metrics.mean_absolute_error(featurePoi.loc[:, 'RANK_LEVEL'], featurePoi['pre_rank_level']))

    except Exception as e:
        print(e)



def convert_level(y):
    if y < 1.5:
        y = 1
    elif y < 2.5:
        y = 2
    elif y <3.5:
        y = 3
    elif y <4.5:
        y = 4
    else:
        y = 5
    return y



if __name__ == '__main__':
    files_rank_cluster('D:\structure\\featurePoi\\featuretest','D:\structure\\featurePoi\\temp_result')

2017-07-08 12:10:24,480 - all fileList handle process


2017-07-08 12:10:24,592 - total file:1
